In [47]:
import datetime
import logging
import os
import numpy as np
from ocean_navigation_simulator.reinforcement_learning.runners.GenerationRunner import (
    GenerationRunner,
)
import matplotlib.pyplot as plt
## Only when developing with VSCode in my repo 
#os.chdir('/home/nicolas/codeRepo/OceanPlatformControl') # here in Azure
# os.chdir('/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl') # here in WSL
# print(os.getcwd())
##
import pandas as pd
from ocean_navigation_simulator.utils.units import Distance
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
wandb_log_name = "30_platforms_on_HC_HC_flocking_0_25"

### Define Filenames

In [49]:
path_to_files = 'generated_media/metrics_csv_results/'
#file_containing_pb = 'problemsGOM.csv'
pb_name_hj_naive = '30_platforms_HC_HC_hj_naive_1191.csv'
pb_name_reactive = '30_platforms_HC_HC_reactive_control_1193.csv'
pb_name_flocking = '30_platforms_HC_HC_flocking_0_25_1173.csv'

### Plot problem distribution

In [50]:
# TODO

### Extract metrics from csv files

In [51]:
metrics_hj_naive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_hj_naive}")
metrics_hj_reactive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_reactive}")
metrics_hj_flocking_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_flocking}")

+ Make sure we have the (same) indices and total number of missions for the panda dataframes

In [52]:
# Find the common mission_nr values
common_mission_nr = set(metrics_hj_naive_unfiltered['mission_nr']).intersection(set(metrics_hj_reactive_unfiltered['mission_nr'])).intersection(set(metrics_hj_flocking_unfiltered['mission_nr']))
# Filter the rows in each dataframe that have matching mission_nr values
metrics_hj_naive = metrics_hj_naive_unfiltered[metrics_hj_naive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_reactive = metrics_hj_reactive_unfiltered[metrics_hj_reactive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_flocking = metrics_hj_flocking_unfiltered[metrics_hj_flocking_unfiltered['mission_nr'].isin(common_mission_nr)]

+ Sort by ascending missions nr order

In [53]:
metrics_hj_naive = metrics_hj_naive.sort_values(['mission_nr'])
metrics_hj_reactive= metrics_hj_reactive.sort_values(['mission_nr'])
metrics_hj_flocking = metrics_hj_flocking.sort_values(['mission_nr'])

+ Index by mission nr to get the same order in the panda dataframe between controller metrics

In [54]:
metrics_hj_naive.set_index('mission_nr', inplace=True)
metrics_hj_reactive.set_index('mission_nr', inplace=True)
metrics_hj_flocking.set_index('mission_nr', inplace=True)

+ Get Non Feasible Missions 

In [55]:
non_feas_idx = metrics_hj_naive.index[metrics_hj_naive["Reaching_target"] < 1]
ratio_non_feas_miss = len(non_feas_idx)/len(metrics_hj_naive)*100
print("Number of infeasible missions: ", len(non_feas_idx), f"corresponds to {ratio_non_feas_miss} %")

Number of infeasible missions:  117 corresponds to 9.991460290350128 %


+ Drop infeasible missions on HC

In [56]:
metrics_hj_naive.drop(non_feas_idx, axis=0, inplace=True)
metrics_hj_reactive.drop(non_feas_idx, axis=0, inplace=True)
metrics_hj_flocking.drop(non_feas_idx, axis=0, inplace=True)

In [57]:
import plotly.express as px
import plotly.graph_objects as go

## Box Plots

In [58]:
from plotly.offline import  init_notebook_mode
init_notebook_mode(connected=True)

+ Beta Index

In [59]:
beta_index_avg_dic = {"hj_naive": metrics_hj_naive["Average Beta Index"],
                      "hj_reactive": metrics_hj_reactive["Average Beta Index"],
                      "flocking": metrics_hj_flocking["Average Beta Index"]}
beta_index_avg_df = pd.DataFrame(beta_index_avg_dic)
fig_beta = px.box(beta_index_avg_df)
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Average Beta Index over Missions"
)
fig_beta.show()

+ Mean minimum distance to target among all platforms km

In [60]:
mean_min_dist_dict = {"hj_naive": Distance(deg=metrics_hj_naive["Mean minimum distance to target among all platforms in deg"]).km,
                      "hj_reactive": Distance(deg=metrics_hj_reactive["Mean minimum distance to target among all platforms in deg"]).km,
                      "flocking":Distance(deg=metrics_hj_flocking["Mean minimum distance to target among all platforms in deg"]).km}
idx_list = []
mean_min_dist_df = pd.DataFrame(mean_min_dist_dict)
fig_min_dist = px.box(mean_min_dist_df, log_y=True)
mean_trace = go.Scatter(
    x=mean_min_dist_df.columns,
    y=mean_min_dist_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_min_dist.add_trace(mean_trace)
# Add custom x and y labels
fig_min_dist.update_layout(
    xaxis_title="controllers",
    yaxis_title=" [km]",
    title = {'text': "Mean minimum distance to target among all platforms"}
)
fig_min_dist.show()

+ Mean maximum correction from optimal control degrees

In [61]:
opt_ctrl_correction_dict = {"hj_naive": metrics_hj_naive["Mean maximum correction from optimal control degrees"],
                      "hj_reactive": metrics_hj_reactive["Mean maximum correction from optimal control degrees"],
                      "flocking": metrics_hj_flocking["Mean maximum correction from optimal control degrees"]}
opt_ctrl_correction_df = pd.DataFrame(opt_ctrl_correction_dict)
fig_opt_ctrl_correction = px.box(opt_ctrl_correction_df )
# Add custom x and y labels
fig_opt_ctrl_correction.update_layout(
    xaxis_title="controllers",
    yaxis_title="degrees",
    title = {'text': "Control direction correction from optimal input"}
)
fig_opt_ctrl_correction.show()

+ Isolated Platform Metric

In [62]:
ipm_dict= {"hj_naive": metrics_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": metrics_hj_reactive["Isolated_platform_metric"],
                      "flocking": metrics_hj_flocking["Isolated_platform_metric"]}
ipm_df = pd.DataFrame(ipm_dict)
fig_ipm = px.box(ipm_df, log_y=False )
mean_trace = go.Scatter(
    x=ipm_df.columns,
    y=ipm_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_ipm.add_trace(mean_trace)
# Add custom x and y labels
fig_ipm.update_layout(
    xaxis_title="controllers",
    yaxis_title="IPM",
    title = {'text': "Isolated_platform_metric"}
)
fig_ipm.show()

### Get Metrics summary as a dict:

In [63]:
def get_metrics_summary(metrics_file:pd.DataFrame, ctrl_name: str)->dict:
    return {
    "ctrl_name": ctrl_name,
    "avg_beta_index":metrics_file["Average Beta Index"].mean(),
    "avg_isolated_pltf_integral": metrics_file["Isolated_platform_metric"].mean(),
    "avg_correction_from_opt_ctrl_deg": metrics_file["Mean maximum correction from optimal control degrees"].mean(),
    "avg_mean_min_dist_to_target_km": Distance(deg=metrics_file["Mean minimum distance to target among all platforms in deg"].mean()).km,
    "avg_nb_platfroms_reaching_target": metrics_file["Reaching_target"].mean(),
    "nb_missions_with_isolated_pltf": np.sum(metrics_file["Isolated_platform_metric"] > 0),
    "nb_missions_with_collisions": np.sum(metrics_file["Number_of_collision"] > 0),
    "nb_missions_success": np.sum(metrics_file["Mission_success"]==1),
}

  + HJ Multi-Agent Naive

In [64]:
metrics_summary_hj_naive = get_metrics_summary(metrics_hj_naive, ctrl_name="hj_naive")
print(metrics_summary_hj_naive)
summary_hj_naive_df = pd.DataFrame(metrics_summary_hj_naive, index=[0])

{'ctrl_name': 'hj_naive', 'avg_beta_index': 4.6784467283090745, 'avg_isolated_pltf_integral_hj_naive': 0.4142339880994213, 'avg_correction_from_opt_ctrl_deg': 0.0, 'avg_mean_min_dist_to_target_km': 0.0, 'avg_nb_platfroms_reaching_target': 1.0, 'nb_missions_with_isolated_pltf': 699, 'nb_missions_with_collisions': 884, 'nb_missions_success': 31}


+ HJ Multi-Agent Decentralized Reactive Control

In [65]:
metrics_summary_hj_reactive = get_metrics_summary(metrics_hj_reactive, ctrl_name='hj_reactive')
print(metrics_summary_hj_reactive)
summary_hj_naive_reactive = pd.DataFrame(metrics_summary_hj_reactive, index=[0])

{'ctrl_name': 'hj_reactive', 'avg_beta_index': 4.390338886999887, 'avg_isolated_pltf_integral_hj_naive': 0.2358889269566847, 'avg_correction_from_opt_ctrl_deg': 79.40869152039197, 'avg_mean_min_dist_to_target_km': 0.09756316982429306, 'avg_nb_platfroms_reaching_target': 0.9774826059456042, 'nb_missions_with_isolated_pltf': 538, 'nb_missions_with_collisions': 0, 'nb_missions_success': 416}


+ HJ Multi-Agent Flocking

In [66]:
metrics_summary_hj_flocking = get_metrics_summary(metrics_hj_flocking, ctrl_name='hj_flocking')
print(metrics_summary_hj_flocking)
summary_hj_naive_flocking= pd.DataFrame(metrics_summary_hj_flocking, index=[0])

{'ctrl_name': 'hj_flocking', 'avg_beta_index': 11.573707575142684, 'avg_isolated_pltf_integral_hj_naive': 0.054762354463888295, 'avg_correction_from_opt_ctrl_deg': 134.63449672399028, 'avg_mean_min_dist_to_target_km': 7.627928515321502, 'avg_nb_platfroms_reaching_target': 0.5639468690702087, 'nb_missions_with_isolated_pltf': 139, 'nb_missions_with_collisions': 328, 'nb_missions_success': 284}


## Confusion Matrices Plots

In [67]:
def plot_confusion_matrix(x, y, x_label, y_label, title_name:str):
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm

    confusion_matrix = pd.crosstab(x, y, rownames=[x_label], colnames=[y_label], normalize=True)*100
    fig = px.imshow(confusion_matrix, text_auto=True)
    fig.update_traces(texttemplate='%{text:.1f}%', textfont_size=12, text=confusion_matrix.values)
    #fig = px.imshow(confusion_matrix.round(2), text_auto=True)
    fig.update_layout(
        title = {'text': title_name}
    )
    fig.show()
    # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, fmt='g')
    # # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, norm=LogNorm(), fmt='g') # if heat map is in norm
    # plt.tick_params(axis='both', which='major', labelbottom=False, bottom=False, top=True, labeltop=True)
    # ax.xaxis.set_label_position('top')
    # ax.plot
    # plt.title(title_name)
    # plt.show()
    return fig

###  HJ naive vs Decentralized Reactive Control

+  Missions with collisions

In [68]:
fig_conf_react_coll = plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_reactive["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with collisions")

+ Missions with communication losses

In [69]:
fig_conf_react_comm_loss= plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_reactive["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [70]:
connect_objective_reactive = (metrics_hj_reactive["Isolated_platform_metric"]==0) & (metrics_hj_reactive["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_react_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_reactive , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Reactive Control', title_name= 'Missions without collisions and with maintained connectivity')

+  Missions success counts

In [71]:
fig_react_success  = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_reactive["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive', title_name="Mission Success")

###  HJ naive vs Flocking

+  Missions with collisions

In [72]:
fig_conf_flocking_coll= plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_flocking["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with collisions")

+ Missions with communication losses

In [73]:
fig_conf_flocking_comm_loss = plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_flocking["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [74]:
connect_objective_flocking = (metrics_hj_flocking["Isolated_platform_metric"]==0) & (metrics_hj_flocking["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_flocking_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_flocking , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Flocking Control', title_name= 'Missions without collisions and with maintained connectivity')

In [75]:
index_list = metrics_hj_flocking.index[(connect_objective_reactive==True) & (connect_objective_flocking==False)].tolist()


In [76]:
index_list

[1,
 2,
 22,
 41,
 45,
 62,
 73,
 74,
 90,
 93,
 101,
 109,
 123,
 131,
 136,
 139,
 147,
 182,
 184,
 191,
 194,
 208,
 211,
 216,
 224,
 233,
 234,
 235,
 236,
 251,
 252,
 262,
 275,
 279,
 292,
 293,
 295,
 306,
 316,
 326,
 335,
 338,
 348,
 364,
 371,
 376,
 396,
 401,
 437,
 438,
 441,
 451,
 453,
 455,
 470,
 471,
 477,
 478,
 483,
 488,
 489,
 495,
 505,
 518,
 529,
 544,
 560,
 564,
 571,
 579,
 583,
 589,
 610,
 620,
 623,
 625,
 629,
 633,
 640,
 646,
 661,
 663,
 668,
 682,
 688,
 691,
 707,
 719,
 720,
 742,
 744,
 753,
 773,
 783,
 786,
 790,
 791,
 798,
 803,
 811,
 816,
 824,
 838,
 858,
 861,
 865,
 867,
 876,
 877,
 878,
 882,
 890,
 902,
 904,
 913,
 916,
 917,
 922,
 924,
 926,
 942,
 943,
 946,
 951,
 960,
 967,
 972,
 974,
 977,
 991,
 993,
 996,
 998,
 1000,
 1008,
 1019,
 1024,
 1035,
 1043,
 1050,
 1051,
 1068,
 1069,
 1070,
 1080,
 1083,
 1085,
 1087,
 1091,
 1092,
 1117,
 1123,
 1139,
 1141,
 1149,
 1154,
 1163,
 1176]

In [77]:
metrics_hj_reactive.loc[285]

Unnamed: 0                                                    781.000000
Isolated_platform_metric                                        0.000000
Number_of_collision                                             0.000000
Reaching_target                                                 1.000000
Mean minimum distance to target among all platforms in deg      0.000000
Mean maximum correction from optimal control degrees          103.818438
Average Beta Index                                              6.389904
Initial maximum degree of the graph                             9.000000
Final maximum degree of the graph                              24.000000
Mission_success                                                 1.000000
mean solver time in seconds                                     2.081122
std solver time in seconds                                      0.298092
Name: 285, dtype: float64

+  Missions success counts

In [78]:
fig_flocking_success = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_flocking["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking', title_name="Mission Success")

## Log Data in WandB

In [79]:
import wandb

In [80]:
# Log metrics in WandB
os.environ["WANDB_API_KEY"] = "1f19232e6ccc9981a8a972bee18ba31a94644835"

wandb.init(
    # Set the project where this run will be logged
    project="Master_Thesis_Run_Summaries",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=wandb_log_name,
    # Track hyperparameters and run metadata
    config={
        "Number of infeasible missions": len(non_feas_idx),
        "Percentage of infeasible missions": ratio_non_feas_miss,
        "Number of missions feasible": len(metrics_hj_naive),
        "Ratio failing missions with HJ naive in terms of connectivity and collisions": 
        len(metrics_hj_naive.index[(connect_objective_hj_naive==False)].tolist())/len(metrics_hj_naive),
    },
    entity="nhoischen",
)
wandb.log({"Metrics_Summary": wandb.Table(dataframe=pd.concat([summary_hj_naive_df, summary_hj_naive_reactive, summary_hj_naive_flocking]))},
            commit=False)
wandb.log({'Beta Index': fig_beta,
          'Time-Integral-over-Isolated-Platforms': fig_ipm,
          'Platforms-Mean-Min-Distance-to-Target': fig_min_dist,
           'Deviation-from-Cptimal-Ctrl-Direction': fig_opt_ctrl_correction,
            },
           commit=False)
wandb.log({'Missions with collisions hj-reactive': fig_conf_react_coll,
         'Missions with collisions hj-flocking': fig_conf_flocking_coll,
         'Missions with communication losses hj-reactive': fig_conf_react_comm_loss,
         'Missions with communication losses hj-flocking': fig_conf_flocking_comm_loss,
         'Missions with maintained connectivity and no collisions hj-reactive': fig_react_maint,
         'Missions with maintained connectivity and no collisions hj-flocking': fig_flocking_maint,
         'Succesfull Missions hj-reactive': fig_react_success,
         'Succesfull Missions hj-flocking': fig_flocking_success}, commit=True)
wandb.finish()